In [1]:
# Block 1 — Imports & Paths
import pandas as pd
import re
from pathlib import Path
from rapidfuzz import process, fuzz

try:
    import camelot
    _HAS_CAMELOT = True
except Exception:
    _HAS_CAMELOT = False
    print("Camelot not installed or Ghostscript missing. Please install it.")

BASE_DIR = Path.cwd().parent  # assumes notebooks/ inside prototype/
RAW_DOCS = BASE_DIR / "data" / "raw" / "documents"
OUTPUT_EXCEL = BASE_DIR / "data" / "processed" / "excel"
OUTPUT_MERGED = BASE_DIR / "data" / "processed" / "text"
MAPPING_FILE = BASE_DIR / "data" / "external" / "district_mapping.csv"

OUTPUT_EXCEL.mkdir(parents=True, exist_ok=True)
OUTPUT_MERGED.mkdir(parents=True, exist_ok=True)



In [2]:
mapping_df = pd.read_csv(MAPPING_FILE, dtype=str).fillna("")
mapping_df.columns = [c.strip().lower() for c in mapping_df.columns]

def norm_text(s):
    s = (s or "")
    s = s.strip().lower()
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

for col in ["block", "village", "district", "state"]:
    if col in mapping_df.columns:
        mapping_df[col + "_norm"] = mapping_df[col].apply(norm_text)
    else:
        mapping_df[col + "_norm"] = ""


In [3]:
def extract_pdf_to_excel(pdf_path: Path, out_dir: Path) -> Path:
    """Extract tables from PDF into Excel using Camelot, try both flavors."""
    out_path = out_dir / f"{pdf_path.stem}.xlsx"
    if out_path.exists():
        print(f"⏩ Skipping {pdf_path.name}, Excel already exists.")
        return out_path

    if not _HAS_CAMELOT:
        raise RuntimeError("Camelot not available")

    # First try lattice
    tables = camelot.read_pdf(str(pdf_path), pages="all", flavor="lattice")

    # If lattice fails or produces too few columns, try stream
    if not tables or max(len(t.df.columns) for t in tables) < 3:
        tables = camelot.read_pdf(str(pdf_path), pages="all", flavor="stream")

    if not tables:
        print(f"❌ No tables found in {pdf_path.name}")
        return None

    dfs = [t.df for t in tables]
    df_all = pd.concat(dfs, ignore_index=True)

    df_all.to_excel(out_path, index=False)
    print(f"✅ Extracted {len(df_all)} rows from {pdf_path.name} → {out_path.name}")
    return out_path


In [4]:
def clean_excel(file_path: Path) -> pd.DataFrame:
    """Clean an extracted Excel into block | village | beneficiary."""
    df = pd.read_excel(file_path, dtype=str).fillna("")
    df.columns = [str(c).strip().lower() for c in df.columns]

    # Detect columns
    block_col = next((c for c in df.columns if "block" in c), None)
    village_col = next((c for c in df.columns if "village" in c), None)
    benef_col = next((c for c in df.columns if any(k in c for k in ["beneficiary", "patta", "holder", "name"])), None)

    block_col = block_col or df.columns[-2] if len(df.columns) > 1 else df.columns[0]
    village_col = village_col or df.columns[-1] if len(df.columns) > 1 else ""
    benef_col = benef_col or df.columns[0]

    df_clean = pd.DataFrame({
        "block": df[block_col].astype(str).str.strip(),
        "village": df[village_col].astype(str).str.strip() if village_col else "",
        "beneficiary": df[benef_col].astype(str).str.strip()
    })

    # Drop junk rows (headers, footers, repeated "Name of the District" etc.)
    df_clean = df_clean[~df_clean["beneficiary"].str.contains(
        "sl\.? no|name of|format for submission|district", case=False, na=False
    )]

    # Remove empty rows
    df_clean = df_clean[(df_clean["beneficiary"] != "") & (df_clean["beneficiary"].str.lower() != "nan")]

    # Deduplicate
    df_clean = df_clean.drop_duplicates()

    return df_clean




<>:23: SyntaxWarning: invalid escape sequence '\.'
<>:23: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Vivek\AppData\Local\Temp\ipykernel_21404\1465625069.py:23: SyntaxWarning: invalid escape sequence '\.'
  "sl\.? no|name of|format for submission|district", case=False, na=False


In [5]:
all_dfs = []

# Step 1: Extract PDF → Excel (skip if already exists)
for pdf in RAW_DOCS.glob("*.pdf"):
    excel_path = extract_pdf_to_excel(pdf, OUTPUT_EXCEL)
    if excel_path:
        df_norm = clean_excel(excel_path)
        all_dfs.append(df_norm)



✅ Extracted 2926 rows from AngulFRABenefLists.pdf → AngulFRABenefLists.xlsx
✅ Extracted 2468 rows from BalasoreFRABeneficiaries.pdf → BalasoreFRABeneficiaries.xlsx
✅ Extracted 6033 rows from GanjamFRABeneficiariesList_2.pdf → GanjamFRABeneficiariesList_2.xlsx


In [6]:
# Block — Extract block (col 1), village (col 3), beneficiary (col 4)
def extract(file_path: Path) -> pd.DataFrame:
    """
    Extract only Block (col 1), Village (col 3), Beneficiary (col 4)
    from a fixed-structure Excel file.
    """
    df = pd.read_excel(file_path, header=None, dtype=str).fillna("")

    # Extract required columns
    df_clean = pd.DataFrame({
        "block": df.iloc[:, 1].str.strip(),
        "village": df.iloc[:, 3].str.strip(),
        "beneficiary": df.iloc[:, 4].str.strip()
    })

    # Remove junk rows (headers, repeated labels, numbers only, etc.)
    mask = (
        df_clean.apply(lambda row: any(
            re.search(r"(sl\.? no|name of|format for submission|district|taluk)", str(x), re.I)
            for x in row.values
        ), axis=1)
        | df_clean.apply(lambda row: all(str(x).isdigit() or str(x).lower() in ["nan", ""] for x in row.values), axis=1)
    )
    df_clean = df_clean[~mask]

    # Deduplicate
    df_clean = df_clean.drop_duplicates()

    return df_clean



In [7]:
def fix_village_beneficiary(df):
    """Split rows where village+beneficiary got merged into one cell (no mapping needed)."""
    fixed_rows = []

    for _, row in df.iterrows():
        village_val = str(row["village"]).strip()
        benef_val = str(row["beneficiary"]).strip()

        # Case 1: Beneficiary missing, village has multiple words
        if not benef_val and len(village_val.split()) > 1:
            parts = village_val.split(maxsplit=1)
            row["village"] = parts[0]
            row["beneficiary"] = parts[1] if len(parts) > 1 else ""

        # Case 2: Both filled, but village has multiple words
        elif benef_val and len(village_val.split()) > 1:
            parts = village_val.split(maxsplit=1)
            row["village"] = parts[0]
            row["beneficiary"] = (parts[1] + " " + benef_val).strip() if len(parts) > 1 else benef_val

        fixed_rows.append(row)

    return pd.DataFrame(fixed_rows)


In [8]:
def fuzzy_match_block(block_name, mapping_blocks, threshold=75):
    """
    Find the closest block from mapping_blocks using fuzzy string match.
    Returns best match if similarity >= threshold, else None.
    """
    if not block_name or str(block_name).lower() in ["nan", ""]:
        return None
    
    match, score, _ = process.extractOne(
        block_name, 
        mapping_blocks, 
        scorer=fuzz.token_sort_ratio
    )
    return match if score >= threshold else None

In [9]:
def add_state_district_fuzzy(df, mapping_df, threshold=85):
    df = df.copy()
    df["block_norm"] = df["block"].apply(norm_text)

    # Deduplicate mapping_df by block_norm (keep first)
    mapping_df = (
        mapping_df.assign(block_norm=mapping_df["block"].apply(norm_text))
        .drop_duplicates(subset=["block_norm"])
    )

    # Create dictionary
    mapping_dict = mapping_df.set_index("block_norm")[["state", "district"]].to_dict(orient="index")

    mapping_blocks = list(mapping_dict.keys())

    state_list, district_list = [], []

    for block in df["block_norm"]:
        if block in mapping_dict:  # exact match
            state_list.append(mapping_dict[block]["state"])
            district_list.append(mapping_dict[block]["district"])
        else:  # fuzzy match
            best = fuzzy_match_block(block, mapping_blocks, threshold)
            if best:
                state_list.append(mapping_dict[best]["state"])
                district_list.append(mapping_dict[best]["district"])
            else:
                state_list.append(None)
                district_list.append(None)

    df["state"] = state_list
    df["district"] = district_list

    return df[["state", "district", "block", "village", "beneficiary"]]




In [10]:
all_clean = []
for excel in OUTPUT_EXCEL.glob("*.xlsx"):
    clean_df = extract(excel)
    clean_df = fix_village_beneficiary(clean_df)
    clean_df = add_state_district_fuzzy(clean_df, mapping_df, threshold=85)  # 👈 fuzzy backtracking
    all_clean.append(clean_df)

merged_clean = pd.concat(all_clean, ignore_index=True)

# Save outputs
merged_clean.to_csv(OUTPUT_MERGED / "fra.csv", index=False)

print("✅ Final rows with fuzzy state+district mapping:", len(merged_clean))




✅ Final rows with fuzzy state+district mapping: 10621
